In [1]:
from gramformer import Gramformer
import gradio as gr

from happytransformer import HappyTextToText, TTSettings

/home/elearning/miniconda3/envs/corrector-env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Gramformer Model implemented in Gradio 

In [ ]:
gf = Gramformer(models=1, use_gpu=False) #1=corrector, 2=detector

In [ ]:
gf.correct('hello my dear childs')

    Function to build web application using gradio

In [ ]:
def correct(sentence):
    res = gf.correct(sentence) 
    return res
app_inputs = gr.inputs.Textbox(lines=3, placeholder="Enter a grammatically incorrect sentence here...")

interface = gr.Interface(fn=correct, 
                        inputs=app_inputs,
                         outputs='text', 
                        title='Hi there, I\'m Gramformer')

#interface.launch()

    Gramformer highlighter

In [ ]:
gh = Gramformer(models=3, use_gpu=False) #1=corrector, 2=detector

In [ ]:
gh.highlight(orig='to tha store',cor='to the store')

## Bert + Huggingface Solution

In [ ]:
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
args = TTSettings(num_beams=5, min_length=1)
result = happy_tt.generate_text("grammar: This sentences has has bads grammar.", args=args)
print(result.text)

In [ ]:
def huggingface_corrector(text):
    result = happy_tt.generate_text('grammar: ' + text, args=TTSettings(num_beams=1, min_length=1, max_length=100))
    return result.text

app_hugginface_inputs = gr.inputs.Textbox(lines=3, placeholder="Enter a grammatically incorrect sentence here...")

interface2 = gr.Interface(fn=huggingface_corrector, 
                        inputs=app_hugginface_inputs,
                        outputs='text', 
                        title='Hi there, I\'m Huggingface')

#interface2.launch()